Ноздрова В.

ИУ5-81Б

#Задача 1

In [ ]:
Variant<-15
set.seed(Variant)
P<-sample(c(4, 6),1)
V<-sample(c("V1", "V2", "V3", "V4"),1)
if ((V=="V3") | (V=="V4")){
  b<-sort(sample(c(1:10),P))
}
lambda<-runif(P)
mu<-runif(P,2,4)
View(data.frame(P,V))
if ((V=="V3") | (V=="V4")) {View(data.frame(lambda,mu, b))}
if ((V=="V1") | (V=="V2")) {View(data.frame(lambda,mu))}

P,V
<dbl>,<chr>
4,V3


lambda,mu,b
<dbl>,<dbl>,<int>
0.8151934,2.209339,1
0.2539684,3.292302,2
0.6872308,3.018181,6
0.8314290,3.413257,10


По теореме Кобхэма:

$\lambda=\sum_{p=1}^P\lambda_p$

$\frac{1}{\mu}=\sum_{p=1}^P \frac{\lambda_p}{\lambda}\cdot \frac{1}{\mu_p}$

$\rho_p=\frac{\lambda_p}{\mu_p} $

$\rho=\frac{\lambda}{\mu}=\sum_{p=1}^P \rho_p$

$W_0=\sum_{p=1}^P\frac{\rho_p}{\mu_p}$

In [ ]:
Lambda <- sum(lambda); Lambda
Mu <- 1/sum(lambda/(Lambda*mu));Mu
rho <- lambda/mu
Rho <- sum(rho); Rho
W0 <- sum(rho/mu); W0

[1] 2.587822

[1] 2.820817

[1] 0.9174015

[1] 0.3372451

$0\leq\rho<1: W_p=\frac{\frac{W_0}{1-\rho}-\sum_{i=1}^{p-1}\rho_i\cdot W_i\cdot(1-b_i/b_p)}{1-\sum_{i=p+1}^P\rho_i\cdot(1-b_p/b_i)}$

In [ ]:
W <- rep(0,P)

for (p in 1:P)
{
  num<-0
  den<-0
  if ((p-1)>=1) {
  for (i in 1:(p-1))
    num <- num+rho[i]*W[i]*(1-b[i]/b[p])}
  num <- W0/(1-Rho) - num
  if ((p+1)<=P) {
  for (i in (p+1):P)
    den <- den+rho[i]*(1-b[p]/b[i]) }
  den <- 1 - den
  W[p] <- num/den
}
W

[1] 7.390570 4.162467 1.768768 1.210721

In [ ]:
install.packages("simmer")
library(simmer)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘BH’




In [ ]:
SIM_TIME <- 500

bs <- b * 100

syst <- simmer("syst")

signal <- "done"

recompute <- trajectory() %>%
          set_prioritization(function() (c((now(syst)-get_attribute(syst,"start"))*get_attribute(syst, "b"),NA,F))) %>%
          rollback(2)

req <- trajectory("req's path") %>%
   set_attribute("start", function() now(syst)) %>%
   renege_if(signal, recompute) %>%
   seize("server") %>%
   renege_abort() %>%
   timeout_from_attribute("timeout") %>%
   set_capacity("server", 0) %>%
   send(signal) %>%
   timeout(0) %>%
   release("server")%>%
   set_capacity("server", 1)

syst %>%
   add_resource("server", mon=T,capacity=1, preemptive=F) %>% invisible

   for (i in 1:P)
   syst %>% add_generator(paste0("req",i," "),
            join(trajectory()%>%
                 set_attribute("type", i)%>%
                 set_attribute("timeout", function() rexp(1, mu[i]))%>%
                 set_attribute("b", bs[i]),
                 req),
            from(function() rexp(1, lambda[i]), function() rexp(1, lambda[i])), mon=2, priority=0) ;

syst %>% run(until = SIM_TIME) %>% invisible

In [ ]:
stts <- get_mon_arrivals(syst, per_resource = T)
stts <- aggregate(stts[,sapply(stts,is.numeric)],stts["name"],sum)
stts <- transform(stts,queue = end_time - start_time - activity_time)
exp <- rep(0,P)
for (i in 1:P) {
   exp[i] <- mean(stts[substr(stts$name, 1, 4)==paste0("req",i),]$queue)
   print(exp[i])}

[1] 7.139504
[1] 4.18394
[1] 1.569511
[1] 1.036535


In [ ]:
df <- data.frame(c(1:P),W, exp)
colnames(df) <-c("type","theory", "experim")
df

type,theory,experim
<int>,<dbl>,<dbl>
1,7.390570,7.139504
2,4.162467,4.183940
3,1.768768,1.569511
4,1.210721,1.036535
